In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib

# === Load Dataset ===
df = pd.read_csv("bog_features1.csv") 
X = df.drop(columns=["label"])
y = df["label"]

# === Label Mapping (adjust as per your label encoding) ===
label_mapping = {
    0: 'cipher',
    1: 'hash',
    2: 'hnv',
    3: 'hnvor',
    4: 'iv',
    5: 'key',
    6: 'tm',
    7: 'tls'
}

# === Split Data ===
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# === Evaluation Function ===
def evaluate_model(model_path, scaler_path, model_name):
    print(f"\n🧪 Evaluating {model_name}")

    # Load model and scaler
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)

    # Scale test data
    X_test_scaled = scaler.transform(X_test)

    # Predict
    y_pred = model.predict(X_test_scaled)

    # === Metrics ===
    acc = accuracy_score(y_test, y_pred)
    print(f"🎯 Accuracy: {round(acc * 100, 2)}%")

    print("\n📋 Classification Report:")
    print(classification_report(
        y_test, y_pred,
        target_names=[label_mapping[i] for i in sorted(label_mapping.keys())]
    ))

    print("📉 Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

# === Evaluate Models ===
evaluate_model("svm_best_model.pkl", "scaler.pkl", "WITHOUT Augmentation")
evaluate_model("svm_aug_better_model.pkl", "scaler_aug_better.pkl", "WITH Augmentation")



🧪 Evaluating WITHOUT Augmentation
🎯 Accuracy: 83.99%

📋 Classification Report:
              precision    recall  f1-score   support

      cipher       0.82      0.88      0.85       296
        hash       0.88      0.91      0.89       306
         hnv       0.82      0.90      0.86       306
       hnvor       0.96      0.59      0.73        39
          iv       0.86      0.73      0.79       107
         key       0.85      0.65      0.74       110
          tm       0.81      0.83      0.82       262
         tls       0.87      0.80      0.84       142

    accuracy                           0.84      1568
   macro avg       0.86      0.79      0.81      1568
weighted avg       0.84      0.84      0.84      1568

📉 Confusion Matrix:
[[260   3  15   0   3   3  10   2]
 [  5 277  10   0   4   0   7   3]
 [  6  12 275   0   1   1   9   2]
 [  4   1   4  23   0   0   6   1]
 [ 10   5   4   0  78   6   4   0]
 [ 15   7   8   0   4  72   4   0]
 [ 11   9  14   0   1   0 218   9]
 [  

In [8]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 1. Load CSV
df = pd.read_csv('bog_features1.csv', header=None)

# 2. Remove invalid label rows (e.g. 'label' string)
df = df[df.iloc[:, -1] != 'label']

# 3. Separate features and labels
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# 4. Encode string labels (for consistency in later predictions)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
joblib.dump(le, "label_encoder_no_aug.pkl")  # Save label encoder

# 5. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 6. Feature scaling (important for MLP)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
joblib.dump(scaler, "scaler_no_aug.pkl")  # Save the scaler

# 7. Define MLP model
mlp = MLPClassifier(
    hidden_layer_sizes=(256, 128, 64),
    activation='relu',
    solver='adam',
    alpha=0.001,
    learning_rate='adaptive',
    max_iter=1000,
    early_stopping=True,
    n_iter_no_change=10,
    validation_fraction=0.1,
    random_state=42,
    verbose=True
)

# 8. Train the model
mlp.fit(X_train, y_train)

# 9. Save the model
joblib.dump(mlp, "mlp_model_no_aug.pkl")
print("✅ Model and scaler saved!")

# 10. Evaluate the model
y_pred = mlp.predict(X_test)
print("🎯 Accuracy:", accuracy_score(y_test, y_pred))
print("📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Iteration 1, loss = 1.36315357
Validation score: 0.654459
Iteration 2, loss = 0.89120268
Validation score: 0.707006
Iteration 3, loss = 0.76170528
Validation score: 0.754777
Iteration 4, loss = 0.67753921
Validation score: 0.778662
Iteration 5, loss = 0.62613402
Validation score: 0.778662
Iteration 6, loss = 0.58692074
Validation score: 0.791401
Iteration 7, loss = 0.56083536
Validation score: 0.805732
Iteration 8, loss = 0.56601344
Validation score: 0.800955
Iteration 9, loss = 0.50834024
Validation score: 0.826433
Iteration 10, loss = 0.47758281
Validation score: 0.818471
Iteration 11, loss = 0.46350987
Validation score: 0.826433
Iteration 12, loss = 0.43914921
Validation score: 0.831210
Iteration 13, loss = 0.43796174
Validation score: 0.802548
Iteration 14, loss = 0.41939134
Validation score: 0.821656
Iteration 15, loss = 0.40969092
Validation score: 0.845541
Iteration 16, loss = 0.38686609
Validation score: 0.839172
Iteration 17, loss = 0.36454212
Validation score: 0.831210
Iterat

In [9]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE

# 1. Load CSV
df = pd.read_csv('bog_features1.csv', header=None)

# 2. Remove invalid label row (with literal string 'label')
df = df[df.iloc[:, -1] != 'label']

# 3. Separate features and labels
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# 4. Encode labels (so SMOTE works and it's consistent)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
joblib.dump(le, "label_encoder_aug.pkl")  # Save encoder

# 5. Train-test split (before SMOTE)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 6. Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
joblib.dump(scaler, "scaler_aug.pkl")  # Save the scaler

# 7. Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_aug, y_train_aug = smote.fit_resample(X_train, y_train)

# 8. Define MLP model
mlp = MLPClassifier(
    hidden_layer_sizes=(256, 128, 64),
    activation='relu',
    solver='adam',
    alpha=0.001,
    learning_rate='adaptive',
    max_iter=1000,
    early_stopping=True,
    n_iter_no_change=10,
    validation_fraction=0.1,
    random_state=42,
    verbose=True
)

# 9. Train the model
mlp.fit(X_train_aug, y_train_aug)

# 10. Save model
joblib.dump(mlp, "mlp_model_aug.pkl")
print("✅ SMOTE-augmented model and scaler saved!")

# 11. Evaluate the model
y_pred = mlp.predict(X_test)
print("🎯 Accuracy:", accuracy_score(y_test, y_pred))
print("📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Iteration 1, loss = 1.25324528
Validation score: 0.685714
Iteration 2, loss = 0.78948519
Validation score: 0.710204
Iteration 3, loss = 0.65606862
Validation score: 0.780612
Iteration 4, loss = 0.56839821
Validation score: 0.780612
Iteration 5, loss = 0.50759067
Validation score: 0.818367
Iteration 6, loss = 0.46517990
Validation score: 0.825510
Iteration 7, loss = 0.43604650
Validation score: 0.835714
Iteration 8, loss = 0.39509006
Validation score: 0.843878
Iteration 9, loss = 0.37196138
Validation score: 0.848980
Iteration 10, loss = 0.35283980
Validation score: 0.839796
Iteration 11, loss = 0.34027737
Validation score: 0.852041
Iteration 12, loss = 0.31021727
Validation score: 0.860204
Iteration 13, loss = 0.29655192
Validation score: 0.859184
Iteration 14, loss = 0.29119521
Validation score: 0.862245
Iteration 15, loss = 0.26516688
Validation score: 0.868367
Iteration 16, loss = 0.25297317
Validation score: 0.881633
Iteration 17, loss = 0.23592923
Validation score: 0.887755
Iterat

In [10]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib

# === Label Mapping ===
label_mapping = {
    0: 'cipher',
    1: 'hash',
    2: 'hnv',
    3: 'hnvor',
    4: 'iv',
    5: 'key',
    6: 'tm',
    7: 'tls'
}

# === Load and split dataset ===
df = pd.read_csv("bog_features1.csv")
X = df.drop(columns=["label"]).astype(float)
y = df["label"]

# No encoding needed; labels already numeric
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# === Generalized evaluation function for SVM ===
def evaluate_and_predict(model_path, scaler_path, model_name):
    print(f"\n🔎 Model: {model_name}")

    # Load model and scaler
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)

    # === Random sample prediction ===
    random_idx = random.randint(0, len(X_test) - 1)
    input_vector = X_test.iloc[random_idx].values.reshape(1, -1)
    true_label_encoded = y_test.iloc[random_idx]
    true_label_name = label_mapping[int(true_label_encoded)]

    input_scaled = scaler.transform(input_vector)
    pred_encoded = model.predict(input_scaled)[0]
    pred_label_name = label_mapping[int(pred_encoded)]

    print(f"🎯 Random Sample Index: {random_idx}")
    print(f"✅ True Label: {true_label_name} (encoded: {true_label_encoded})")
    print(f"🔍 Predicted Label: {pred_label_name} (encoded: {pred_encoded})")

    # === Full test set evaluation ===
    X_test_scaled = scaler.transform(X_test)
    y_pred = model.predict(X_test_scaled)

    print("\n📊 Test Set Evaluation:")
    print(f"🎯 Accuracy: {round(accuracy_score(y_test, y_pred) * 100, 2)}%")
    print("📋 Classification Report:")
    print(classification_report(y_test, y_pred, target_names=[label_mapping[i] for i in sorted(label_mapping)]))
    print("📉 Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

# === Run for both models ===
evaluate_and_predict("svm_best_model.pkl", "scaler.pkl", "SVM WITH Augmentation")
evaluate_and_predict("svm_aug_better_model.pkl", "scaler_aug_better.pkl", "SVM WITHOUT Augmentation")



🔎 Model: SVM WITH Augmentation
🎯 Random Sample Index: 1538
✅ True Label: tm (encoded: 6)
🔍 Predicted Label: tm (encoded: 6)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



📊 Test Set Evaluation:
🎯 Accuracy: 83.99%
📋 Classification Report:
              precision    recall  f1-score   support

      cipher       0.82      0.88      0.85       296
        hash       0.88      0.91      0.89       306
         hnv       0.82      0.90      0.86       306
       hnvor       0.96      0.59      0.73        39
          iv       0.86      0.73      0.79       107
         key       0.85      0.65      0.74       110
          tm       0.81      0.83      0.82       262
         tls       0.87      0.80      0.84       142

    accuracy                           0.84      1568
   macro avg       0.86      0.79      0.81      1568
weighted avg       0.84      0.84      0.84      1568

📉 Confusion Matrix:
[[260   3  15   0   3   3  10   2]
 [  5 277  10   0   4   0   7   3]
 [  6  12 275   0   1   1   9   2]
 [  4   1   4  23   0   0   6   1]
 [ 10   5   4   0  78   6   4   0]
 [ 15   7   8   0   4  72   4   0]
 [ 11   9  14   0   1   0 218   9]
 [  7   2   4   

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



📊 Test Set Evaluation:
🎯 Accuracy: 82.08%
📋 Classification Report:
              precision    recall  f1-score   support

      cipher       0.78      0.85      0.81       296
        hash       0.81      0.89      0.85       306
         hnv       0.84      0.87      0.85       306
       hnvor       0.74      0.59      0.66        39
          iv       0.90      0.78      0.83       107
         key       0.79      0.67      0.73       110
          tm       0.85      0.79      0.82       262
         tls       0.84      0.76      0.80       142

    accuracy                           0.82      1568
   macro avg       0.82      0.78      0.79      1568
weighted avg       0.82      0.82      0.82      1568

📉 Confusion Matrix:
[[253  14  11   2   2   6   7   1]
 [ 11 272  11   0   2   1   4   5]
 [ 13  12 267   1   0   2   8   3]
 [  5   2   3  23   2   0   2   2]
 [  7   5   2   0  83   5   5   0]
 [ 15   8   6   3   2  74   2   0]
 [ 12  15  13   1   1   3 207  10]
 [  9   7   6   

In [11]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib

# === Label Mapping (for display only) ===
label_mapping = {
    0: 'cipher',
    1: 'hash',
    2: 'hnv',
    3: 'hnvor',
    4: 'iv',
    5: 'key',
    6: 'tm',
    7: 'tls'
}

# === Load and preprocess dataset ===
df = pd.read_csv("bog_features1.csv", header=None)
df = df[df.iloc[:, -1] != 'label']
X = df.iloc[:, :-1].astype(float).values
y = df.iloc[:, -1].values

# Load label encoder
label_encoder = joblib.load("label_encoder_aug.pkl")  # assuming both encoders have same mapping
y_encoded = label_encoder.transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, stratify=y_encoded, test_size=0.2, random_state=42)

# === Generalized evaluation function for MLP ===
def evaluate_mlp(model_path, scaler_path, encoder_path, model_name):
    print(f"\n🔎 Model: {model_name}")

    # Load model, scaler, encoder
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    encoder = joblib.load(encoder_path)

    # === Random sample selection ===
    random_idx = random.randint(0, len(X_test) - 1)
    input_vector = X_test[random_idx].reshape(1, -1)
    true_label_encoded = y_test[random_idx]
    true_label_name = encoder.inverse_transform([true_label_encoded])[0]

    # Prediction
    input_scaled = scaler.transform(input_vector)
    pred_encoded = model.predict(input_scaled)[0]
    pred_label_name = encoder.inverse_transform([pred_encoded])[0]

    print(f"🎯 Random Sample Index: {random_idx}")
    print(f"✅ True Label: {true_label_name} (encoded: {true_label_encoded})")
    print(f"🔍 Predicted Label: {pred_label_name} (encoded: {pred_encoded})")

    # === Evaluation on test set ===
    X_test_scaled = scaler.transform(X_test)
    y_pred = model.predict(X_test_scaled)

    print("\n📊 Test Set Evaluation:")
    print(f"🎯 Accuracy: {round(accuracy_score(y_test, y_pred) * 100, 2)}%")
    print("📋 Classification Report:")
    print(classification_report(y_test, y_pred, target_names=encoder.classes_))
    print("📉 Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

# === Run for both models ===
evaluate_mlp("mlp_model_no_aug.pkl", "scaler_no_aug.pkl", "label_encoder_no_aug.pkl", "MLP WITHOUT Augmentation")
evaluate_mlp("mlp_model_aug.pkl", "scaler_aug.pkl", "label_encoder_aug.pkl", "MLP WITH Augmentation")



🔎 Model: MLP WITHOUT Augmentation
🎯 Random Sample Index: 404
✅ True Label: 0 (encoded: 0)
🔍 Predicted Label: 0 (encoded: 0)

📊 Test Set Evaluation:
🎯 Accuracy: 80.42%
📋 Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.84      0.82       296
           1       0.89      0.85      0.87       306
           2       0.78      0.88      0.83       306
           3       0.83      0.51      0.63        39
           4       0.67      0.73      0.70       107
           5       0.79      0.59      0.68       110
           6       0.82      0.78      0.80       262
           7       0.79      0.81      0.80       142

    accuracy                           0.80      1568
   macro avg       0.80      0.75      0.77      1568
weighted avg       0.81      0.80      0.80      1568

📉 Confusion Matrix:
[[250   6  19   0  12   2   5   2]
 [  7 261  14   1   8   2   6   7]
 [  9   8 268   1   4   2  11   3]
 [  3   1   3  20   4   2   5  

In [12]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib

# === Label Mapping (for display only) ===
label_mapping = {
    0: 'cipher',
    1: 'hash',
    2: 'hnv',
    3: 'hnvor',
    4: 'iv',
    5: 'key',
    6: 'tm',
    7: 'tls'
}

# === Load and preprocess dataset ===
df = pd.read_csv("bog_features1.csv", header=None)
df = df[df.iloc[:, -1] != 'label']
X = df.iloc[:, :-1].astype(float).values
y = df.iloc[:, -1].values

# Load label encoder
label_encoder = joblib.load("label_encoder_aug.pkl")  # assuming both encoders have same mapping
y_encoded = label_encoder.transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, stratify=y_encoded, test_size=0.2, random_state=42)

# === Generalized evaluation function for MLP ===
def evaluate_mlp(model_path, scaler_path, encoder_path, model_name):
    print(f"\n🔎 Model: {model_name}")

    # Load model, scaler, encoder
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    encoder = joblib.load(encoder_path)

    # === Random sample selection ===
    random_idx = random.randint(0, len(X_test) - 1)
    input_vector = X_test[random_idx].reshape(1, -1)
    true_label_encoded = y_test[random_idx]
    true_label_name = encoder.inverse_transform([true_label_encoded])[0]

    # Prediction
    input_scaled = scaler.transform(input_vector)
    pred_encoded = model.predict(input_scaled)[0]
    pred_label_name = encoder.inverse_transform([pred_encoded])[0]

    print(f"🎯 Random Sample Index: {random_idx}")
    print(f"✅ True Label: {true_label_name} (encoded: {true_label_encoded})")
    print(f"🔍 Predicted Label: {pred_label_name} (encoded: {pred_encoded})")

    # === Evaluation on test set ===
    X_test_scaled = scaler.transform(X_test)
    y_pred = model.predict(X_test_scaled)

    print("\n📊 Test Set Evaluation:")
    print(f"🎯 Accuracy: {round(accuracy_score(y_test, y_pred) * 100, 2)}%")
    print("📋 Classification Report:")
    print(classification_report(y_test, y_pred, target_names=encoder.classes_))
    print("📉 Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

# === Run for both models ===
evaluate_mlp("mlp_model_no_aug.pkl", "scaler_no_aug.pkl", "label_encoder_no_aug.pkl", "MLP WITHOUT Augmentation")
evaluate_mlp("mlp_model_aug.pkl", "scaler_aug.pkl", "label_encoder_aug.pkl", "MLP WITH Augmentation")



🔎 Model: MLP WITHOUT Augmentation
🎯 Random Sample Index: 1227
✅ True Label: 7 (encoded: 7)
🔍 Predicted Label: 7 (encoded: 7)

📊 Test Set Evaluation:
🎯 Accuracy: 80.42%
📋 Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.84      0.82       296
           1       0.89      0.85      0.87       306
           2       0.78      0.88      0.83       306
           3       0.83      0.51      0.63        39
           4       0.67      0.73      0.70       107
           5       0.79      0.59      0.68       110
           6       0.82      0.78      0.80       262
           7       0.79      0.81      0.80       142

    accuracy                           0.80      1568
   macro avg       0.80      0.75      0.77      1568
weighted avg       0.81      0.80      0.80      1568

📉 Confusion Matrix:
[[250   6  19   0  12   2   5   2]
 [  7 261  14   1   8   2   6   7]
 [  9   8 268   1   4   2  11   3]
 [  3   1   3  20   4   2   5 